# Running the Micom Workflow for the Binary Azotobacter - Synechococcus Model

In this notebook we utilize the package `micom` to generate a binary community model for 2 organisms of interest:
- `Azotobacter vinelandii`
- `Synechococcus elongatus`

This binary consortium allows us to gain insights into the exchanges between the 2 organisms and run FBA experiments.

First off we can import all necessary packages for this notebook.

In [ ]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium,build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

import os
os.environ["GRB_LICENSE_FILE"]

In [ ]:
import requests
def load_model_from_git(model_name: str):
    """Helper function for loading a SBML compliant model into CobraPy directly from Git

    Args:
        model_name (str): The name of the model you wish to load

    Returns:
        model (cobra.core.model.Model): A Cobra Model object

    """
    
    model_url_dict = dict([('Rhodosporidium','https://raw.githubusercontent.com/PNNL-CompBio/RToruGEM/main/rtoru/Rhodo_Toru.xml'),
                           ('Azotobacter','https://raw.githubusercontent.com/PNNL-CompBio/iAzotobacterVinelandiiGEM/main/a_vine/azo_vine.xml'),
                           ('Synechococcus','https://raw.githubusercontent.com/PNNL-CompBio/S-elongatus7942/main/syn_elong/syn_elong.xml')])

    xml = requests.get(model_url_dict[model_name])

    with open('tmp.xml', 'wb') as f:
        f.write(xml.content)
        model = cobra.io.read_sbml_model('tmp.xml')
        
    return model

In [ ]:
models = ['Rhodosporidium', 'Synechococcus']
m_names = ['Rhodo_Toru.xml', 'syn_elong.xml']

for i,j in zip(models, m_names):
    cobra.io.write_sbml_model(load_model_from_git(i), j)

## Setting Up the Model in MICOM

To begin, we need to import our genome-scale models into `micom`. We have these models saved as `.sbml` files as this form is accepted easily into programs such as `cobrapy` and `micom`.

### Building a Taxonomy

Step #1: Establish a Taxonomy that lists the out the taxonomy for our organisms of interest

In [ ]:
Tax= pd.DataFrame(columns=['id','genus','species','reactions','metabolites','sample_id','abundance'])
Tax.loc[len(Tax.index)] = ['Synechococcus', 'Synechococcus','S. elongatus',851,769,'One',500]
Tax.loc[len(Tax.index)] = ['Azotobacter','Azotobacter','A. vinelandii',2469,2003,'One',500]
Tax

This taxonomy file lists important information for `micom` down the road, such as the number of `reactions` and `metabolites` in the provided models.

### Building a Database

Step #2: Now we must construct a database for the `.sbml` models to be preprocessed and stored. This is done by supplying `micom` with a file which contains model path locations.

In [ ]:
db = pd.read_csv('../Manifest_Files/man_av_se.csv')
db['file'] = m_names

In [ ]:
build_database(db,'./db_av_se')

In [ ]:
db_path = './db_av_se' 

### Construct Manifest Object

Step #3: Now that we have the Taxonomy and Database constructed we can build our community model. This is done by using the `build()` method in `micom`.

__Note__:

Build Manifest object from Taxonomy DataFrame and the corresponding database directory

Skip this step if manifest has already been built and saved to "models" directory


__IMPORTANT__: Declare the Solver you would like to use for this Community model here:
- osqp (good for smaller models)
- gurobi
- glpk
- cplex
- scipy


In [ ]:
manifest = build(Tax, out_folder="models_av_se", model_db=db_path, cutoff=0.0001, threads=10,solver='gurobi')
manifest

## Running the Models with FBA

Now that we have the `manifest`, we can load the model as a `Community` object through `micom`. This will give us some functionality similar to that of `cobrapy`. This can be done with the `load_pickle()` method we imported above through `micom`.

In [ ]:
community = load_pickle("models_av_se/One.pickle")
print(len(community.reactions))

### Exploring the Model Attributes

Our new variable `community` behaves very similarly to a standard `cobrapy` model. We can explore it's attributes in a similar way as well.

Things such as `reactions` and `metabolites`:

In [ ]:
community.reactions

In [ ]:
community.metabolites

and importantly the `medium`

In [ ]:
community.medium

This behavior mimics the medium in `cobrapy`, but combines both models mediums into 1

### Running Optimization

Now that we have the model loaded, we can run standard `FBA` methods using `optimize()`. Default optimize does not return any fluxes from the model, so we can set the `fluxes=True` when calling the method to return them.

In [ ]:
result = community.optimize(fluxes=True)
result

We can see that both organisms have a non-zero growth rate and that the community growth is also non-zero. Let's check the fluxes.

In [ ]:
result.fluxes.T.loc['EX_nh4_e']

#### Changing parts of the medium to test it's effect on growth

Now that we can successfully optimize the community model, we can begin altering the models medium and seeing how it changes the (community) growth rate.

First, let's make a copy of the original medium so that we can restore it after making changes.

In [ ]:
medium_bkp = community.medium

Now we can make changes to the medium. The following cell is meant to be re-run with making changes. It will first restore the medium to the original and them set 

In [ ]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_nh4_m"] = 0
#medium_to_change["EX_sucr_m"] = 0
#medium_to_change["EX_leu__L_m"] = 1

# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

Now that the medium is changed, we can rerun the model optimization.

In [ ]:
result_altered_medium = community.optimize(fluxes=True)

In [ ]:
result_altered_medium.fluxes.T.loc['EX_nh4_e']

## Running the Models with MICOM Grow

An alternative to running standard community optimization with `optimize()`, we can also use a `micom.workflows` method called `grow()`. This simulates growth of the organism while also simulating potential tradeoffs (between prioritizing community vs. individual growth). This method does not require our previously constructed `community` object, but rather the `manifest` we added earlier.

A key difference here though, is that we need to create a `DataFrame` detailing the reaction, flux, and metabolite as the medium provided to the method.

### Building the Medium

In [ ]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
grow_medium_to_change = community.medium

#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
#grow_medium_to_change["EX_sucr_m"] = 1
grow_medium_to_change["EX_nh4_m"] = 0


In [ ]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

In [ ]:
result_grow = grow(manifest, model_folder="models_av_se", medium=grow_medium, tradeoff=0.01, threads=2,presolve=True)

In [ ]:
result_grow.exchanges

In [ ]:
result_grow.exchanges.to_csv('av_se_out.csv')